In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.6 MB/s eta 0:00:00


In [3]:
import ast
import numpy as np
from tqdm import tqdm
from transformers import pipeline
import pandas as pd
import re
import string
import collections

In [4]:
# Metric calculation taken from https://rajpurkar.github.io/SQuAD-explorer/
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [5]:
question_answerer = pipeline("question-answering", model="/home/vp.shivasan/interiit/task2/training_dir/task2_50epochs_2e-5_FINAL_ES/checkpoint-2538")

OSError: ignored

In [ ]:
my_df = pd.read_csv('/home/vp.shivasan/interiit/data/Task2dataSet_test.csv')
F1s = []
for i in tqdm(range(len(my_df))):
    question = my_df['question'][i]
    context = my_df['context'][i]
    correct_answer = ast.literal_eval(my_df['answers'][i])['text'][0]
    predicted_answer = question_answerer(question=question, context=context)['answer']
    F1s.append(compute_f1(correct_answer,predicted_answer))

print("Average F1 score",np.mean(F1s))
